In [ ]:
import os 

In [ ]:
storage_account = os.environ["AZURE_STORAGE_ACCOUNT_NAME"]
access_key = os.environ["AZURE_STORAGE_ACCOUNT_ACCESS_KEY"]
NESSIE_URI = os.environ['NESSIE_URI']
REF = "main"
FULL_PATH_TO_WAREHOUSE = os.environ['WAREHOUSE']
AWS_S3_ENDPOINT = os.environ["AWS_S3_ENDPOINT"]
AWS_ACCESS_KEY = os.environ['AWS_ACCESS_KEY_ID']
AWS_SECRET_KEY  = os.environ['AWS_SECRET_ACCESS_KEY']

In [ ]:
from pyspark import SparkConf

conf = SparkConf() \
    .setAppName("process_sales_silver") \
    .setMaster("local[*]") \
    .set("spark.executor.memory", "4g") \
    .set("spark.driver.memory", "2g") \
    

# set minio config
conf.set("spark.sql.catalog.nessie.s3.endpoint", AWS_S3_ENDPOINT)
# conf.set("spark.sql.catalog.nessie.warehouse", FULL_PATH_TO_WAREHOUSE)
# conf.set("spark.sql.catalog.nessie.uri", NESSIE_URI)
conf.set("spark.sql.catalog.nessie.ref", REF)
conf.set("spark.sql.catalog.nessie.authentication.type", "NONE") 
conf.set("spark.sql.catalog.nessie.s3.access-key-id", AWS_ACCESS_KEY)
conf.set("spark.sql.catalog.nessie.s3.secret-access-key", AWS_SECRET_KEY)
conf.set("spark.sql.catalog.nessie.s3.path-style-access", "true")



In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.config(conf=conf).getOrCreate()
spark

# Process sales header

In [ ]:
spark.sql("TRUNCATE TABLE nessie.silver.sales_header")

In [ ]:
spark.sql("""
WITH CTE_SALES AS
(
  select 
      SalesOrderNumber
      , CustomerUsername
      , OrderDate
      , DueDate
      , ShipDate
      , SalesTerritoryRegion
      , Currency
      , source_filepath
      , ingestion_datetime
      , row_number() over(partition by SalesOrderNumber, CustomerUsername, OrderDate, DueDate, ShipDate, SalesTerritoryRegion, Currency order by 1) as rownum
    from nessie.adventureWorks.sales
)
MERGE INTO nessie.silver.sales_header as target
USING (
  select *
  from CTE_SALES
  where rownum = 1
) as source
ON target.SalesOrderNumber = source.SalesOrderNumber
WHEN MATCHED THEN 
UPDATE SET 
  target.SalesOrderNumber = source.SalesOrderNumber
  , target.CustomerUsername = source.CustomerUsername
  , target.OrderDate = source.OrderDate
  , target.DueDate = source.DueDate
  , target.ShipDate = source.ShipDate
  , target.SalesTerritoryRegion = source.SalesTerritoryRegion
  , target.Currency = source.Currency
  , target.source_filepath = source.source_filepath
  , target.ingestion_datetime = source.ingestion_datetime
WHEN NOT MATCHED THEN INSERT
(
  SalesOrderNumber
  , CustomerUsername
  , OrderDate
  , DueDate
  , ShipDate
  , SalesTerritoryRegion
  , Currency
  , source_filepath
  , ingestion_datetime
)
VALUES
(
  source.SalesOrderNumber
  , source.CustomerUsername
  , source.OrderDate
  , source.DueDate
  , source.ShipDate
  , source.SalesTerritoryRegion
  , source.Currency
  , source.source_filepath
  , source.ingestion_datetime
)
""").show()

In [ ]:
%%sql 

select *
from nessie.silver.sales_header